# **Import Libraries and modules**

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [2]:
import sys
import numpy
import string
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences


from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
filename = "/content/gdrive/My Drive/Colab Notebooks/EIP 3/wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
#raw_text = raw_text.translate(None, string.punctuation)

In [0]:
punctuations = '''!()-[]{};:'"\,<>/?@#$%^&*_~'''

#my_str = "Hello!!!, he said ---and went."

# remove punctuation from the string
no_punct = ""
for char in raw_text:
   if char not in punctuations:
       no_punct = no_punct + char

# display the unpunctuated string
raw_text = no_punct
raw_text1 = keras.preprocessing.text.hashing_trick(raw_text, 100, hash_function=None, filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t', lower=True, split='.')

In [47]:
print(raw_text1)

{0: 81, 1: 11, 2: 51, 3: 19, 4: 57, 5: 80, 6: 37, 7: 49, 8: 49, 9: 3, 10: 28, 11: 5, 12: 56, 13: 3, 14: 75, 15: 31, 16: 99, 17: 16, 18: 68, 19: 1, 20: 16, 21: 86, 22: 90, 23: 60, 24: 98, 25: 75, 26: 59, 27: 8, 28: 31, 29: 53, 30: 60, 31: 29, 32: 60, 33: 77, 34: 21, 35: 48, 36: 41, 37: 95, 38: 40, 39: 54, 40: 86, 41: 80, 42: 50, 43: 12, 44: 28, 45: 52, 46: 92, 47: 49, 48: 59, 49: 76, 50: 11, 51: 92, 52: 74, 53: 76, 54: 86, 55: 65, 56: 60, 57: 84, 58: 27, 59: 81, 60: 49, 61: 95, 62: 9, 63: 7, 64: 45, 65: 60, 66: 13, 67: 27, 68: 20, 69: 77, 70: 51, 71: 56, 72: 64, 73: 34, 74: 60, 75: 21, 76: 34, 77: 9, 78: 68, 79: 63, 80: 98, 81: 9, 82: 8, 83: 82, 84: 6, 85: 57, 86: 60, 87: 21, 88: 95, 89: 93, 90: 68, 91: 77, 92: 87, 93: 30, 94: 61, 95: 7, 96: 56, 97: 55, 98: 55, 99: 24, 100: 29, 101: 31, 102: 21, 103: 89, 104: 8, 105: 61, 106: 25, 107: 94, 108: 56, 109: 37, 110: 35, 111: 66, 112: 81, 113: 72, 114: 31, 115: 47, 116: 15, 117: 20, 118: 60, 119: 11, 120: 60, 121: 55, 122: 18, 123: 41, 124: 6

In [0]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
chars1 = sorted(list(set(raw_text1)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [26]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

n_chars1 = len(raw_text1)
n_vocab1 = len(chars1)
print ("Total Characters: ", n_chars1)
print ("Total Vocab: ", n_vocab1)

Total Characters:  142352
Total Vocab:  32
Total Characters:  2087
Total Vocab:  99


In [50]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	#print(seq_in)
	#seq_in = pad_sequences(seq_in, maxlen=100) #Padded sequence
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  142252


In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# define the LSTM model
model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
#model.add(Dropout(0.1))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# define the checkpoint
filepath="/content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [54]:
model.fit(X, y, epochs=100, batch_size=256, callbacks=callbacks_list)

Epoch 1/100
142252/142252 [==============================] - 73s 510us/step - loss: 2.8825

Epoch 00001: loss improved from inf to 2.88250, saving model to /content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5
Epoch 2/100
142252/142252 [==============================] - 72s 505us/step - loss: 2.7718

Epoch 00002: loss improved from 2.88250 to 2.77179, saving model to /content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5
Epoch 3/100
142252/142252 [==============================] - 72s 507us/step - loss: 2.7024

Epoch 00003: loss improved from 2.77179 to 2.70235, saving model to /content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5
Epoch 4/100
142252/142252 [==============================] - 72s 505us/step - loss: 2.6565

Epoch 00004: loss improved from 2.70235 to 2.65653, saving model to /content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5
Epoch 5/100
142252/142252 [==============================] - 72s 506us/step - loss: 2.6149

Epoch 00005: loss improved from 

In [0]:
# load the network weights
filename = "/content/gdrive/My Drive/Colab Notebooks/EIP 3/assign2.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [57]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
" and turns out his toes.

              later editions continued as follows
    when the sands are al "
l to yeel and the selter wiat soe eonled at eoulousoe io she hrr thmerken.

  `how sheay oo toe se the seid te the waic to herself athe wuon of the herter and the whmnght in  oo tie fraligot and the wei  `o ce rateerer the sioule lild to the and the wai  `o ce ruoeree in serty.nf herself in soment bnd the whmught tt herself and the wem onhh the led thait hear she wooe tie whou sie heard i could to  here whe soece uf the hoass rot aelon her hnn she her and the whing to beant  andce what she wioug
Done.
